In [1]:
%%capture

from google.colab import drive, files
drive.mount('/content/drive/')
%cd /content/drive/My Drive/AI/NewsCrawl

In [2]:
import pandas as pd
import numpy as np
from glob import glob
import pandas as pd
import numpy as np
import re
import sys
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier

In [3]:
# Loại bỏ warning

import logging
for name in logging.Logger.manager.loggerDict.keys():
    logging.getLogger(name).setLevel(logging.CRITICAL)

# Load dữ liệu từ dataset

In [4]:
dataset = []
dataset.extend(glob('./vnexpress-dantri-merged/*.xlsx'))
dataset

['./vnexpress-dantri-merged/thoi-su.xlsx',
 './vnexpress-dantri-merged/the-gioi.xlsx',
 './vnexpress-dantri-merged/kinh-doanh.xlsx',
 './vnexpress-dantri-merged/the-thao.xlsx',
 './vnexpress-dantri-merged/giai-tri.xlsx',
 './vnexpress-dantri-merged/phap-luat.xlsx',
 './vnexpress-dantri-merged/suc-khoe.xlsx',
 './vnexpress-dantri-merged/giao-duc.xlsx',
 './vnexpress-dantri-merged/khoa-hoc-cong-nghe.xlsx',
 './vnexpress-dantri-merged/du-lich.xlsx',
 './vnexpress-dantri-merged/oto-xe-may.xlsx',
 './vnexpress-dantri-merged/train.xlsx',
 './vnexpress-dantri-merged/test.xlsx']

In [5]:
train_df = pd.DataFrame()
test_df = pd.DataFrame()
train_df = train_df.append(pd.read_excel('./vnexpress-dantri-merged/train.xlsx'), ignore_index=True)
test_df = test_df.append(pd.read_excel('./vnexpress-dantri-merged/test.xlsx'), ignore_index=True)
train_df.dropna(inplace = True) # Loại bỏ null
test_df.dropna(inplace = True) # Loại bỏ null

In [6]:
train_df.head()

,Unnamed: 0,id,title,content,label
0,0,2fdc17c5-3abc-4a32-b565-4f9a31b0b63f,Đề xuất xây đường kéo dài đại lộ Thăng Long,"Ngày 17/5, đại diện Bộ Giao thông Vận tải cho ...",0
1,1,4fa7b6b0-5969-4927-9392-d9981971d3e3,Đề xuất xây thêm bãi đỗ xe tại sân bay Nội Bài,Theo đề xuất của Cục Hàng không Việt Nam gửi B...,0
2,2,ef592a33-23e7-496e-a7bd-0cef5f4b3cd2,Đề xuất xóa 4 tổng cục thuộc Bộ Nông nghiệp,"Theo dự thảo nghị định quy định chức năng, nhi...",0
3,3,3a699019-f1a8-4d25-8427-1f1e62329521,Đề xuất xử lý hình sự hành vi thu gom sổ bảo h...,"Kiến nghị được Giám đốc Sở Lao động, Thương bi...",0
4,4,582a4158-78be-44a0-b7e9-9019abfbf1ad,Đêm hội trăng rằm của trẻ có hoàn cảnh khó khăn,Chương trình mở cửa miễn phí với nhiều tiết mụ...,0


In [7]:
test_df.head()

,Unnamed: 0,id,title,content,label
0,0,efde56c2-b0d4-4aef-91dd-874058c186ac,1.000 ha dừa Bến Tre nhiễm sâu đầu đen,"Sáng gần giữa tháng tư, ông Nguyễn Long Hổ (50...",0
1,1,26af923d-7550-4d59-a5cd-1ccfb2e988ff,1.000 túi thuốc hỗ trợ F0 điều trị tại nhà,"Sáng 17/9, 1.000 túi thuốc đóng gói sẵn được c...",0
2,2,9bc5be8f-460f-46e7-b418-3e3a3464e43e,1/4 số thực tập sinh Nhật Bản trở về Việt Nam ...,Cơ quan Hợp tác Quốc tế Nhật Bản (JICA) vừa cô...,0
3,3,adfad0b4-7924-483e-ac26-2e12de121ca0,"1,5 triệu người chưa nhận, TP HCM kéo dài hỗ t...","Chiều 22/10, Giám đốc Sở Lao động, Thương binh...",0
4,4,2e4ab952-2a50-41fc-a32b-f40c51495157,1.500 tỷ đồng xây mới cầu Trà Khúc 1,"Chiều 19/4, Chủ tịch UBND Quảng Ngãi Đặng Văn ...",0


In [8]:
labels = []
for label in train_df['label']:
    labels.append(label)


labels = list(set(labels))
print(f"Có tổng cộng {len(labels)} nhãn")
print(list(set(labels)))

Có tổng cộng 11 nhãn
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [9]:
id2label = {
    0: 'thời sự',
    1: 'thế giới',
    2: 'kinh doanh',
    3: 'thể thao',
    4: 'giải trí',
    5: 'pháp luật',
    6: 'sức khỏe',
    7: 'giáo dục',
    8: 'khoa học - công nghệ',
    9: 'du lịch - ẩm thực',
    10: 'oto xe máy'
}


label_names = [id2label[i] for i in range(len(labels))]
print(label_names)

['thời sự', 'thế giới', 'kinh doanh', 'thể thao', 'giải trí', 'pháp luật', 'sức khỏe', 'giáo dục', 'khoa học - công nghệ', 'du lịch - ẩm thực', 'oto xe máy']


# Các hàm hỗ trợ

In [10]:
def get_corpus_and_label(df):
    title = [title for title in df['title']]
    content = [content for content in df['content']]
    label = [label for label in df['label']]

    corpus = []
    for index in range(len(title)):
        # merge tiêu đề và nội dung bài viết
        text = title[index] + ' ' + content[index]
        corpus.append(text)
    
    return label, corpus

In [11]:
# Lấy nhãn và corpus từ dataset

train_label, train_corpus = get_corpus_and_label(train_df)
test_label, test_corpus = get_corpus_and_label(test_df)

In [12]:
# sns.set(font_scale=1.4)
# cm = confusion_matrix(y_test, y_pred) 
# cm_df = pd.DataFrame(cm,
#                      index = label_names, 
#                      columns = label_names)

# plt.figure(figsize=(10,8))
# sns.heatmap(cm_df, annot=True, fmt="d",annot_kws={"size": 18})
# plt.ylabel('True label')
# plt.xlabel('Predicted label')
# plt.title('Countvector')
# plt.show()

# Train và Test

## CountVectorizer

In [13]:
cv = CountVectorizer(max_features=5000)

x_train = cv.fit_transform(train_corpus).toarray()
y_train = np.array(train_label)

x_test = cv.transform(test_corpus).toarray()
y_test = np.array(test_label)

In [14]:
# Naive Bayes
# Train
nb_classifier_1 = MultinomialNB()
nb_classifier_1.fit(x_train, y_train)

# Test
y_pred = nb_classifier_1.predict(x_test)
nb1_acc = accuracy_score(y_test, y_pred)

print(nb_classifier_1.__class__.__name__, nb1_acc)

MultinomialNB 0.8129090909090909


In [15]:
# Logistic Regression
# Train
lr_classifier_1 = LogisticRegression(multi_class='multinomial')
lr_classifier_1.fit(x_train, y_train)

# Test
y_pred = lr_classifier_1.predict(x_test)
lr1_acc = accuracy_score(y_test, y_pred)

print(lr_classifier_1.__class__.__name__, lr1_acc)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression 0.8859090909090909


In [16]:
# Decision Tree 
# Train
tree_classifier_1 = DecisionTreeClassifier()
tree_classifier_1.fit(x_train, y_train)

# Test
y_pred = tree_classifier_1.predict(x_test)
tree1_acc = accuracy_score(y_test, y_pred)

print(tree_classifier_1.__class__.__name__, tree1_acc)

DecisionTreeClassifier 0.7191818181818181


In [17]:
# SVM
# Train
svm_classifier_1 = LinearSVC()
svm_classifier_1.fit(x_train, y_train)

# Test
y_pred = svm_classifier_1.predict(x_test)
svm1_acc = accuracy_score(y_test, y_pred)

print(svm_classifier_1.__class__.__name__, svm1_acc)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


LinearSVC 0.8828181818181818


In [18]:
# KNeighborsClassifier
# Train
knn_classifier_1 = KNeighborsClassifier(n_neighbors=3)
knn_classifier_1.fit(x_train, y_train)

# Test
y_pred = knn_classifier_1.predict(x_test)
knn1_acc = accuracy_score(y_test, y_pred)

print(knn_classifier_1.__class__.__name__, knn1_acc)

KNeighborsClassifier 0.8280909090909091


## Tfidf

In [19]:
cv = TfidfVectorizer(max_features=5000)
x_train = cv.fit_transform(train_corpus).toarray()
y_train = np.array(train_label)
x_test = cv.transform(test_corpus).toarray()
y_test = np.array(test_label)

In [20]:
# Naive Bayes
# Train
nb_classifier_2 = MultinomialNB()
nb_classifier_2.fit(x_train, y_train)

# Test
y_pred = nb_classifier_2.predict(x_test)
nb2_acc = accuracy_score(y_test, y_pred)

print(nb_classifier_2.__class__.__name__, nb2_acc)

MultinomialNB 0.8098181818181818


In [21]:
# Logistic Regression
# Train
lr_classifier_2 = LogisticRegression(multi_class='multinomial')
lr_classifier_2.fit(x_train, y_train)

# Test
y_pred = lr_classifier_2.predict(x_test)
lr2_acc = accuracy_score(y_test, y_pred)

print(lr_classifier_2.__class__.__name__, lr2_acc)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression 0.9018181818181819


In [22]:
# Decision Tree 
# Train
tree_classifier_2 = DecisionTreeClassifier()
tree_classifier_2.fit(x_train, y_train)

# Test
y_pred = tree_classifier_2.predict(x_test)
tree2_acc = accuracy_score(y_test, y_pred)

print(tree_classifier_2.__class__.__name__, tree2_acc)

DecisionTreeClassifier 0.7151818181818181


In [23]:
# SVM
# Train
svm_classifier_2 = LinearSVC()
svm_classifier_2.fit(x_train, y_train)

# Test
y_pred = svm_classifier_2.predict(x_test)
svm2_acc = accuracy_score(y_test, y_pred)

print(svm_classifier_2.__class__.__name__, svm2_acc)

LinearSVC 0.9157272727272727


In [33]:
# KNeighborsClassifier
# Train
knn_classifier_2 = KNeighborsClassifier(n_neighbors=3)
knn_classifier_2.fit(x_train, y_train)

# Test
y_pred = knn_classifier_2.predict(x_test)
knn2_acc = accuracy_score(y_test, y_pred)

print(knn_classifier_2.__class__.__name__, knn2_acc)

KNeighborsClassifier 0.8318181818181818


# TỔNG HỢP

In [29]:
board = pd.DataFrame(
    index = ['Naive Bayes', 'Logictic Regression', 'Decision Tree', 'SVM', 'KNN'],
    columns= ['Countvector', 'Tfidf'],
    data= [
              [nb1_acc,nb2_acc],
              [lr1_acc,lr2_acc],
              [tree1_acc, tree2_acc],
              [svm1_acc, svm2_acc],
              [knn1_acc, knn2_acc]
          ]
)

In [30]:
board

,Countvector,Tfidf
Naive Bayes,0.812909,0.809818
Logictic Regression,0.885909,0.901818
Decision Tree,0.719182,0.715182
SVM,0.882818,0.915727
KNN,0.828091,0.831818


# Ensemble

In [27]:
from sklearn.ensemble import VotingClassifier

cv = TfidfVectorizer(max_features=5000)
x_train = cv.fit_transform(train_corpus).toarray()
y_train = np.array(train_label)
x_test = cv.transform(test_corpus).toarray()
y_test = np.array(test_label)


voting_classifier = VotingClassifier( estimators=[
    ('nb1', nb_classifier_1),
    ('nb2', nb_classifier_2),
    # ('lr1', lr_classifier_1),
    ('lr2', lr_classifier_2),
    # ('svm1', svm_classifier_1),
    ('svm2', svm_classifier_2)
    ], voting='hard', weights=[1, 1, 2, 2])

voting_classifier.fit(x_train, y_train)


for clf in (nb_classifier_1,nb_classifier_2,lr_classifier_1, lr_classifier_2, svm_classifier_1, svm_classifier_2, voting_classifier):
    y_pred = clf.predict(x_test)  
    print(clf.__class__.__name__ , accuracy_score(y_test, y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


MultinomialNB 0.801
MultinomialNB 0.8098181818181818
LogisticRegression 0.22736363636363635
LogisticRegression 0.9018181818181819
LinearSVC 0.6121818181818182
LinearSVC 0.9157272727272727
VotingClassifier 0.9022727272727272


In [28]:
# from sklearn.ensemble import AdaBoostClassifier

# adaboost_model = AdaBoostClassifier(base_estimator=svm_classifier_2, n_estimators=50, learning_rate=0.2,algorithm='SAMME')
# adaboost_model.fit(x_train, y_train)
# y_pred = adaboost_model.predict(x_test)
# print(adaboost_model.__class__.__name__ , accuracy_score(y_test, y_pred))